In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("customer_churn_messy.csv")

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64374 entries, 0 to 64373
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CustomerID         64374 non-null  int64 
 1   Age                64374 non-null  int64 
 2   Gender             64374 non-null  object
 3   Tenure             64374 non-null  int64 
 4   Usage Frequency    64374 non-null  int64 
 5   Support Calls      64374 non-null  int64 
 6   Payment Delay      64374 non-null  int64 
 7   Subscription Type  64374 non-null  object
 8   Contract Length    64374 non-null  object
 9   Total Spend        64374 non-null  int64 
 10  Last Interaction   64374 non-null  int64 
 11  Churn              64374 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 5.9+ MB


,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0


In [5]:
df = df.drop_duplicates()

In [6]:
df['Gender'] = (
    df['Gender']
    .str.strip()
    .str.lower()
    .replace({
        'm': 'male',
        'f': 'female'
    })
)

In [7]:
df['Gender'].value_counts()

Gender
female    34353
male      30021
Name: count, dtype: int64

In [8]:
df['Age'] = df['Age'].fillna(df['Age'].median())

In [9]:
Q1 = df['Total Spend'].quantile(0.25)
Q3 = df['Total Spend'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [10]:
df['Total Spend'] = np.clip(df['Total Spend'], lower_bound, upper_bound)

In [11]:
df = df[(df['Total Spend'] >= lower_bound) & (df['Total Spend'] <= upper_bound)]

In [12]:
df = pd.get_dummies(df, columns=['Gender'], drop_first=True)

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_cols = ['Age', 'Total Spend']

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [14]:
df.isnull().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64374 entries, 0 to 64373
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         64374 non-null  int64  
 1   Age                64374 non-null  float64
 2   Tenure             64374 non-null  int64  
 3   Usage Frequency    64374 non-null  int64  
 4   Support Calls      64374 non-null  int64  
 5   Payment Delay      64374 non-null  int64  
 6   Subscription Type  64374 non-null  object 
 7   Contract Length    64374 non-null  object 
 8   Total Spend        64374 non-null  float64
 9   Last Interaction   64374 non-null  int64  
 10  Churn              64374 non-null  int64  
 11  Gender_male        64374 non-null  bool   
dtypes: bool(1), float64(2), int64(7), object(2)
memory usage: 5.5+ MB
